In [1]:
import json
import pandas as pd
import re
with open('data/train/train_data.json', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)
df = df.assign(correct_answer='')
df

,id,Problem,options,Rationale,correct,diagramRef,category,correct_answer
0,14782,"a man whose bowling average is 12.4 , takes 5 ...","a ) 82 , b ) 83 , c ) 84 , d ) 85 , e ) 86","""12.4 x + 26 / ( 5 + x ) = 12 12.4 x + 26 = 60...",d,,general,
1,14783,find the slope of the line perpendicular to th...,"a ) 1 , b ) 2 , c ) - 1 , d ) 4 , e ) 5",two lines are perpendicular if the product of ...,c,,general,
2,14784,a flagstaff 17.5 m high casts a shadow of leng...,"a ) 12.5 m , b ) 10 m , c ) 17.5 m , d ) 21.25...","""let the height of the building x metres . les...",a,,physics,
3,14785,"the length of the bridge , which a train 160 m...","a ) 766 m , b ) 156 m , c ) 215 m , d ) 156 m ...","""speed = ( 45 * 5 / 18 ) m / sec = ( 25 / 2 ) ...",c,,physics,
4,14786,find the simple interest on $ 10000 at 8 % per...,"a ) $ 410 , b ) $ 500 , c ) $ 650 , d ) $ 800 ...","""p = $ 10000 r = 8 % t = 12 / 12 years = 1 yea...",d,,gain,
...,...,...,...,...,...,...,...,...
46027,60809,"Define the sequence $a_1, a_2, a_3, \ldots$ by...",,"By the product-to-sum formula,\n\[\sin \frac{1...",,,precalculus,
46028,60810,Find the number of real solutions of the equat...,,"Since $-1 \le \sin x \le 1,$ all solutions mus...",,,precalculus,
46029,60811,"Let $G$ be the centroid of triangle $ABC,$ and...",,"Let $\mathbf{a}$ denote $\overrightarrow{A},$ ...",,,precalculus,
46030,60812,If angle $A$ lies in the second quadrant and $...,,"Since angle $A$ lies in the second quadrant, $...",,,precalculus,


# Preprocess for multiple choice question

In [2]:
multiple_choice = df[df.options != '']
multiple_choice

,id,Problem,options,Rationale,correct,diagramRef,category,correct_answer
0,14782,"a man whose bowling average is 12.4 , takes 5 ...","a ) 82 , b ) 83 , c ) 84 , d ) 85 , e ) 86","""12.4 x + 26 / ( 5 + x ) = 12 12.4 x + 26 = 60...",d,,general,
1,14783,find the slope of the line perpendicular to th...,"a ) 1 , b ) 2 , c ) - 1 , d ) 4 , e ) 5",two lines are perpendicular if the product of ...,c,,general,
2,14784,a flagstaff 17.5 m high casts a shadow of leng...,"a ) 12.5 m , b ) 10 m , c ) 17.5 m , d ) 21.25...","""let the height of the building x metres . les...",a,,physics,
3,14785,"the length of the bridge , which a train 160 m...","a ) 766 m , b ) 156 m , c ) 215 m , d ) 156 m ...","""speed = ( 45 * 5 / 18 ) m / sec = ( 25 / 2 ) ...",c,,physics,
4,14786,find the simple interest on $ 10000 at 8 % per...,"a ) $ 410 , b ) $ 500 , c ) $ 650 , d ) $ 800 ...","""p = $ 10000 r = 8 % t = 12 / 12 years = 1 yea...",d,,gain,
...,...,...,...,...,...,...,...,...
32027,46809,Eva and Magnus took a road trip and shared the...,"a)70\% , b)75\% , c)80\% , d)85\% , e)90\%",,C,,open,
32028,46810,"Kyanna used three pieces of ribbon, each 8 inc...","a)\(200 - 8g\) , b)\(200 - 2g\) , c)\(200 - g\...",,B,,open,
32029,46811,"Sequence Y: 5, 10, 15, ... Sequence Z: 3, 6, 1...","a)5 , b)6 , c)7 , d)8 , e)9",,B,,closed,
32030,46812,"After 21 students were added to a class, there...","a)3 , b)7 , c)11 , d)14 , e)17",,B,,open,


### For options

In [3]:
def str_to_dict(s):
    s = s.replace('[','').replace(']','').replace("'",'')
    if s == '':
        return ''
    clean_key = lambda x: x.replace("'", '').replace(')', '').strip().lower()
    clean_val = lambda x: x.strip().replace("'", '').replace(']', '').replace('[', '')
    if 'a )' in s:
        items = re.findall(r'([a-zA-Z]+\s*\)) (.+?)(?=,\s*[a-zA-Z]+\s*\)|$)', s)
    else:
        items = re.findall(r'([a-zA-Z]+\s*\))\s*(.+?)(?=,\s*[a-zA-Z]+\s*\)|$)', s)
    return {clean_key(k): clean_val(v) for k, v in items}

for i in range(1000,1010):
    s1 = multiple_choice.options.iloc[i]
    print(str_to_dict(s1).values())


dict_values(['29 / 34', '47 / 100', '3 / 20', '13 / 102', 'none of these'])
dict_values(['24 %', '25 %', '30 %', '33 %', '40 %'])
dict_values(['76 kg', '76.5 kg', '85 kg', '90 kg', 'none'])
dict_values(['240', '388', '379', '277', '320'])
dict_values(['36.7', '52.9', '52.8', '52.1', '52.2'])
dict_values(['5', '17', '16', '23', '19'])
dict_values(['4', '2', '0', '− 2', '− 4'])
dict_values(['65', '100', '115', '130', '260'])
dict_values(['25 sec', '23 sec', '34 sec', '22 sec', '6 sec'])
dict_values(['2 / 3', '1 / 3', '2 / 9', '1 / 12', '1 / 24'])


In [4]:
err_list = []
for i in range(len(multiple_choice)):
    try:
        d = str_to_dict(multiple_choice.options.iloc[i])
        if len(d) < 2:
            err_list.append(i)
            print(multiple_choice.options.iloc[i], d)
    except:
        err_list.append(i)
        print(multiple_choice.options.iloc[i])
print("Error:", len(err_list))

Error: 0


In [5]:
df = df.assign(options_dict = df.options.apply(lambda x: str_to_dict(x)))

In [6]:
df.loc[df.options != '','correct_answer'] = df.loc[df.options != ''].apply(lambda row: row['options_dict'].get(row['correct'].lower()), axis=1)


In [7]:
multiple_choice = df[df.options != '']
multiple_choice[['options', 'correct', 'correct_answer']].head(20)

,options,correct,correct_answer
0,"a ) 82 , b ) 83 , c ) 84 , d ) 85 , e ) 86",d,85
1,"a ) 1 , b ) 2 , c ) - 1 , d ) 4 , e ) 5",c,- 1
2,"a ) 12.5 m , b ) 10 m , c ) 17.5 m , d ) 21.25...",a,12.5 m
3,"a ) 766 m , b ) 156 m , c ) 215 m , d ) 156 m ...",c,215 m
4,"a ) $ 410 , b ) $ 500 , c ) $ 650 , d ) $ 800 ...",d,$ 800
5,"a ) 35 , b ) 40 , c ) 45 , d ) 50 , e ) 55",b,40
6,"a ) 1 / 35 , b ) 1 / 10 , c ) 1 / 3 , d ) 1 / ...",d,1 / 462
7,"a ) 90 % , b ) 30 % , c ) 40 % , d ) 70 % , e ...",a,90 %
8,"a ) 281 m , b ) 325 m , c ) 117 m , d ) 375 m ...",d,375 m
9,"a ) 198 , b ) 364 , c ) 369 , d ) 207 , e ) 210",b,364


### For rationale

In [8]:
def clean_rationale(rationale: str, cutoff: int=25):
    if len(rationale) == 0: return ''
    if rationale[0] == "\"":       rationale = rationale[1:]
    if rationale[-1] == "\"":      rationale = rationale[:-1]
    # Step 1: Delete the last character if it is in [a-e]
    if rationale[-1].lower() in ['a', 'b', 'c', 'd', 'e']: rationale = rationale[:-1]
    # Step 2: Delete from position start with a pattern in patterns
    patterns = [
        "the correct",
        "the ans",
        "final ans",
        "correct",
        "answer",
        "ans ",
        "option",
    ]
    n = len(rationale)
    # Loop and find last conclusion and delete it
    for pattern in patterns:
        pos = rationale.rfind(pattern)
        if 0 <= pos >= n - cutoff:
            rationale = rationale[:pos]
            break
    return rationale

In [9]:
# Xóa các kết luận: 'Correct answer is ...', 'option : ...', 'final answer : ...', ...
df.loc[df.options != '', 'Rationale'] = df.loc[df.options != '', 'Rationale'].apply(clean_rationale)

# Preprocess for fill in blank problems

In [10]:
def extract_ans(text):
    start = text.rfind('\\boxed{')
    while start != -1:
        count = 1
        end = start + 7
        while count > 0 and end < len(text):
            if text[end] == '{':
                count += 1
            elif text[end] == '}':
                count -= 1
            end += 1
        if count == 0:
            ans = text[start+7:end-1]
            for i in ['\left', '\\right', ' ','^\circ']:
                ans = ans.replace(i, '')
            return ans
    return ''

# Extract answer from Rationale column to fill into correct column for fill in blank question without correct answer in dataset
df.loc[(df.options == '') & (df.correct == ''), 'correct'] = df.loc[(df.options == '') & (df.correct == ''), 'Rationale'].apply(extract_ans)


In [12]:
df.loc[df.options == '', 'correct_answer'] = df.loc[df.options == '', 'correct']

In [13]:
# 5 câu có lời giải nhưng trong kết luận của lời giải không có đáp án
df[(df.options == '') & (df.correct == '')]['Rationale']

43914    We multiply both sides of the equation by $10x...
44031    We see that $(x + y)^2 = (x^2 + y^2) + 2xy = 1...
45404    Observe that for all $k \in 1< k< n$, since $k...
45406    First, note that $n$ does not have a prime num...
Name: Rationale, dtype: object

### Clean rationale

In [14]:
fill_blank = df.loc[df.options == '']
fill_blank[['Problem', 'Rationale']]

,Problem,Rationale
30849,"If \(2x^2 - 68 = 10(x + 10)\), what is one pos...",
30855,"In the figure above, what is the value of \(a ...",
30860,"If \(\frac{q}{2} = \frac{7}{10}\), what is the...",
30861,If the perimeter of square WXYZ above is \(16\...,
30864,"Peter can finish a task in 8 hours, and Sam ca...",
...,...,...
46027,"Define the sequence $a_1, a_2, a_3, \ldots$ by...","By the product-to-sum formula,\n\[\sin \frac{1..."
46028,Find the number of real solutions of the equat...,"Since $-1 \le \sin x \le 1,$ all solutions mus..."
46029,"Let $G$ be the centroid of triangle $ABC,$ and...","Let $\mathbf{a}$ denote $\overrightarrow{A},$ ..."
46030,If angle $A$ lies in the second quadrant and $...,"Since angle $A$ lies in the second quadrant, $..."


In [15]:
from bs4 import BeautifulSoup
from markdown import markdown
import re

def markdown_to_text(markdown_string):
    """ Converts a markdown string to plaintext """

    # md -> html -> text since BeautifulSoup can extract text cleanly
    html = markdown(markdown_string)

    # remove code snippets
    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code >', ' ', html)

    # extract text
    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(string=True))

    return text

In [16]:
def remove_asy_content(input_string):
    # Sử dụng biểu thức chính quy để xóa nội dung giữa các thẻ [asy] và [/asy]
    return re.sub(r'\[asy\].*?\[/asy\]', '', input_string, flags=re.DOTALL).strip()

def remove_latex_tags(input_string):
    if input_string == '': return input_string 
    replacements = {
        ' $': ' ', '$ ': ' ', '$-' : '-', '$,': ',', '\$' : '$', '.$' : '.', '$?' : '$', '$.' : '.',
        '\\left': '', '\\right' : '', '\\bullet' : '-', '\\text' : '',
        '\sin' : 'sin', '\cos' : 'cos', '\\tan' : 'tan', '\cot' : 'cot', '\sec' : 'sec', '\csc' : 'csc',
        '\\arcsin' : 'arcsin', '\\arccos' : 'arccos', '\\arctan' : 'arctan', '\\arccot' : 'arccot',
        '\leq' : '<=', '\geq': '>=', '\lt' : '<', '\gt' : '>', '\\neq' : '≠', '\le' : '<=', '\ge' : '>=',
        '\quad': '', '\[' : '', '\]' : '', '&=' : '=',
        '^\circ' : '°', '\mathbf' : '', '\mathbb' : '',
        '\\begin{align}' : '', '\end{align}':'', '\\begin{align*}' : '', '\end{align*}':'',
        '\\times' : '*', '\operatorname' : '', '\displaystyle' : '',
        '\dots':'...', '\ldots' : '...', '\\\n' : '\n', '\cdot' : '*'
    }
    if input_string[-1] == '$': input_string = input_string[:-1]
    for old, new in replacements.items():
        input_string = input_string.replace(old, new)
    return input_string

In [17]:
def clean_fill_blank_problem(problem):
    return remove_latex_tags(remove_asy_content(markdown_to_text(problem))).strip()

def clean_fill_blank_rationale(rationale):
    return clean_fill_blank_problem(rationale).replace('\\boxed', '')

In [18]:
df.loc[df.options == '', 'Problem'] = df.loc[df.options == '', 'Problem'].apply(clean_fill_blank_problem)
df.loc[df.options == '', 'Rationale'] = df.loc[df.options == '', 'Rationale'].apply(clean_fill_blank_rationale)

In [19]:
df.drop(df[(df['options'] == '') & (df['correct'].str.contains('x|y'))].index, inplace=True)

In [20]:
# Save dataframe to json in beautiful format
df.to_json('data/train/train_data_with_correct_answer.json', orient='records', indent=4)


In [21]:
df

,id,Problem,options,Rationale,correct,diagramRef,category,correct_answer,options_dict
0,14782,"a man whose bowling average is 12.4 , takes 5 ...","a ) 82 , b ) 83 , c ) 84 , d ) 85 , e ) 86",12.4 x + 26 / ( 5 + x ) = 12 12.4 x + 26 = 60 ...,d,,general,85,"{'a': '82', 'b': '83', 'c': '84', 'd': '85', '..."
1,14783,find the slope of the line perpendicular to th...,"a ) 1 , b ) 2 , c ) - 1 , d ) 4 , e ) 5",two lines are perpendicular if the product of ...,c,,general,- 1,"{'a': '1', 'b': '2', 'c': '- 1', 'd': '4', 'e'..."
2,14784,a flagstaff 17.5 m high casts a shadow of leng...,"a ) 12.5 m , b ) 10 m , c ) 17.5 m , d ) 21.25...",let the height of the building x metres . less...,a,,physics,12.5 m,"{'a': '12.5 m', 'b': '10 m', 'c': '17.5 m', 'd..."
3,14785,"the length of the bridge , which a train 160 m...","a ) 766 m , b ) 156 m , c ) 215 m , d ) 156 m ...",speed = ( 45 * 5 / 18 ) m / sec = ( 25 / 2 ) m...,c,,physics,215 m,"{'a': '766 m', 'b': '156 m', 'c': '215 m', 'd'..."
4,14786,find the simple interest on $ 10000 at 8 % per...,"a ) $ 410 , b ) $ 500 , c ) $ 650 , d ) $ 800 ...",p = $ 10000 r = 8 % t = 12 / 12 years = 1 year...,d,,gain,$ 800,"{'a': '$ 410', 'b': '$ 500', 'c': '$ 650', 'd'..."
...,...,...,...,...,...,...,...,...,...
46027,60809,"Define the sequence a_1, a_2, a_3, ... by a_n ...",,"By the product-to-sum formula,\n[sin \frac{1}{...",628,,precalculus,628,
46028,60810,Find the number of real solutions of the equat...,,"Since -1 <= sin x <= 1, all solutions must lie...",63,,precalculus,63,
46029,60811,"Let G be the centroid of triangle ABC, and let...",,"Let {a} denote \overrightarrow{A}, etc. Then...",3,,precalculus,3,
46030,60812,If angle A lies in the second quadrant and sin...,,"Since angle A lies in the second quadrant, cos...",-\frac{\sqrt{7}}{4},,precalculus,-\frac{\sqrt{7}}{4},
